In [1]:
import time
import pickle
import numpy as np
import cv2
from mtcnn.mtcnn import MTCNN
from tensorflow.keras.models import load_model
from collections import namedtuple
import numpy as np
import cv2

In [2]:
model = load_model('data/inceptionV3-model.h5')

In [3]:
with open('data/category2label.pkl', 'rb') as pf:
    category2label = pickle.load(pf)

print(category2label)

img_size = (100, 100)
colors = {0: (0, 0, 255), 1: (0, 255, 0)}

{0: 'without_mask', 1: 'with_mask'}


In [4]:
detector = MTCNN()
cap = cv2.VideoCapture(0)



while cap.isOpened():
    ret, frame = cap.read()  
    
    if not ret:
        break
    
    
    #frame=cv2.imread('0.jpg')
    frame = cv2.flip(frame, 1)       # Mirror the image
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    faces = detector.detect_faces(rgb)
    for face in faces:
        try:
            x, y, w, h = face['box'] 
            # Predict
            roi =  rgb[y : y+h, x : x+w]
            data = cv2.resize(roi, img_size)
            data = data / 255.
            data = data.reshape((1,) + data.shape)
            scores = model.predict(data)
            target = np.argmax(scores, axis=1)[0]
            # Draw bounding boxes
            cv2.rectangle(img=frame, pt1=(x, y), pt2=(x+w, y+h), color=colors[target], thickness=2)
            text = "{}: {:.2f}".format(category2label[target], scores[0][target])
            cv2.putText(frame, text, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
        except Exception as e:
            print(e)
            print(roi.shape)

    # Show the frame
    cv2.imshow('Face Mask Detection', frame)
    

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [ ]:
Detection = namedtuple("Detection", ["image_path", "gt", "pred"])
def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou
examples = [
	Detection("0.jpg", [48, 12, 117, 111], [x, y, x+w, y+h]),]
	#Detection("1.jpg", [245, 84, 359, 206], [x, y, x+w, y+h]),]
	#Detection("2.jpg", [187, 60, 378, 265], [x, y, x+w, y+h]),]
for detection in examples:
	# load the image
	image = cv2.imread(detection.image_path)
	# draw the ground-truth bounding box along with the predicted
	# bounding box
	cv2.rectangle(image, tuple(detection.gt[:2]), 
		tuple(detection.gt[2:]), (0, 255, 0), 2)
	cv2.rectangle(image, tuple(detection.pred[:2]), 
		tuple(detection.pred[2:]), (0, 0, 255), 2)
	# compute the intersection over union and display it
	iou = bb_intersection_over_union(detection.gt, detection.pred)
	cv2.putText(image, "IoU: {:.4f}".format(iou), (10, 30),
		cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
	print("{}: {:.4f}".format(detection.image_path, iou))
	# show the output image
	cv2.imshow("Image", image)
	cv2.waitKey(0)

0.jpg: 0.5246
